In [1]:
!pip install git+https://github.com/mlfoundations/open_clip.git


  Cloning https://github.com/mlfoundations/open_clip.git to /tmp/pip-req-build-yswcrciy
  Running command git clone --filter=blob:none --quiet https://github.com/mlfoundations/open_clip.git /tmp/pip-req-build-yswcrciy
  Resolved https://github.com/mlfoundations/open_clip.git to commit a87f11eaf354000d2736580855ae0d9b76ad2a22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━

**No of samples after spliting**

In [3]:
import os
import torch
from torchvision import datasets
from torch.utils.data import random_split, DataLoader
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"


clip_model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-16', 
    pretrained='openai'
)
clip_model.eval().to(device)
for p in clip_model.parameters(): p.requires_grad = False


path = "/kaggle/input/pacs-dataset/kfold/art_painting"
dataset = datasets.ImageFolder(path, transform=preprocess)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])


print(f"Total: {len(dataset)}, Train: {len(train_ds)}, Test: {len(test_ds)}")


train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)


open_clip_model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:442: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Total: 2048, Train: 1638, Test: 410


**"CLIP ViT-B/16 - Art_painting Domain Classification"**

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import open_clip


device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-16', 
    pretrained='openai'
)
clip_model.eval().to(device)
for param in clip_model.parameters():
    param.requires_grad = False 


class CLIPMLPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        self.mlp = nn.Sequential(
            nn.Linear(self.clip.visual.output_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            feats = self.clip.encode_image(x)
        return self.mlp(feats)


def get_train_test_loader(domain, root, batch_size=32, split_ratio=0.8):
    full_ds = datasets.ImageFolder(os.path.join(root, domain), transform=preprocess)
    num_train = int(len(full_ds) * split_ratio)
    num_test = len(full_ds) - num_train
    train_ds, test_ds = random_split(full_ds, [num_train, num_test])
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader, len(full_ds.classes)

PACS_PATH = "/kaggle/input/pacs-dataset/kfold"
target_domain = "art_painting"  

train_loader, test_loader, num_classes = get_train_test_loader(target_domain, PACS_PATH)


model = CLIPMLPClassifier(clip_model, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)


best_acc = 0
wait = 0
patience = 10
best_model_state = None

for epoch in range(50):
    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Training Loss: {total_loss:.4f}")

    # Validation/Test
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test Accuracy on '{target_domain}': {acc:.2f}%")

    
    if acc > best_acc:
        best_acc = acc
        best_model_state = model.state_dict()
        wait = 0
        print("Accuracy improved. Model saved.")
    else:
        wait += 1
        print(f"No improvement. Wait count: {wait}/{patience}")
        if wait >= patience:
            print("Early stopping triggered.")
            break


if best_model_state:
    model.load_state_dict(best_model_state)
    print(f"Best model with {best_acc:.2f}% accuracy loaded.")


Epoch 1 - Training: 100%|██████████| 52/52 [00:26<00:00,  1.97it/s]


Epoch 1 - Training Loss: 89.8468
Test Accuracy on 'art_painting': 79.02%
Accuracy improved. Model saved.


Epoch 2 - Training: 100%|██████████| 52/52 [00:16<00:00,  3.18it/s]


Epoch 2 - Training Loss: 63.8010
Test Accuracy on 'art_painting': 93.66%
Accuracy improved. Model saved.


Epoch 3 - Training: 100%|██████████| 52/52 [00:16<00:00,  3.17it/s]


...

Epoch 24 - Training Loss: 1.6294
Test Accuracy on 'art_painting': 99.51%
No improvement. Wait count: 8/10


Epoch 25 - Training: 100%|██████████| 52/52 [00:16<00:00,  3.11it/s]


Epoch 25 - Training Loss: 1.5422
Test Accuracy on 'art_painting': 99.51%
No improvement. Wait count: 9/10


Epoch 26 - Training: 100%|██████████| 52/52 [00:16<00:00,  3.17it/s]


Epoch 26 - Training Loss: 1.4210
Test Accuracy on 'art_painting': 99.51%
No improvement. Wait count: 10/10
Early stopping triggered.
Best model with 99.51% accuracy loaded.
